In [ ]:
import yfinance as yf
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta, time, timezone
from IPython.display import display
import plotly.express as px
import tdclient as TDClient

In [2]:
df = yf.download('EURUSD=X', start='2025-01-01', interval= '30m')
hourly_df = yf.download('EURUSD=X', start='2025-01-01', interval= '1h')
daily_df = yf.download('EURUSD=X', start='2025-01-01', interval= '1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
ohlc_df = pd.DataFrame(df)
hourly_df = pd.DataFrame(hourly_df)
daily_df = pd.DataFrame(daily_df)
ohlc_df

In [ ]:
hourly_df

In [ ]:
daily_df

In [ ]:
ohlc_df["pivot"] = (daily_df["High"] + daily_df["Low"] + daily_df["Close"]) / 3
ohlc_df["prange"] = (daily_df["High"] - daily_df["Low"])
ohlc_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
ohlc_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
ohlc_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

ohlc_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
ohlc_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
ohlc_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
ohlc_df

In [ ]:
furst_sma_period = 1
second_sma_period = 14
# sma 1
ohlc_df['sma_1'] = hourly_df['Close'].rolling(furst_sma_period).mean()
ohlc_df['prev_sma_1'] = ohlc_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
ohlc_df['sma_14'] = hourly_df['Close'].rolling(second_sma_period).mean()
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
# sma 11
ohlc_df['sma_11'] = daily_df['Close'].rolling(thurd_sma_period).mean()
ohlc_df['prev_sma_11'] = ohlc_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
ohlc_df['sma_114'] = daily_df['Close'].rolling(fourth_sma_period).mean()
ohlc_df

In [ ]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('15:00:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
trades_within_hours = ohlc_df[(ohlc_df.index.time >= start_time) & (ohlc_df.index.time <= end_time)]

# Close all open positions at 11:45 PM
first_trade_per_day = daily_df.groupby(daily_df.index.time).first()
end_trading_time = datetime.strptime('23:45:00', '%H:%M:%S').time()
open_positions = trades_within_hours[trades_within_hours.index.time == end_trading_time]
# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"]
daily_df["buy_stope_loss"] = (daily_df["R1"] - daily_df["S2"]) /2 - daily_df["S2"]
daily_df

In [ ]:
if daily_df["High"] >= daily_df["R2"] and open_positions and first_trade_per_day and ohlc_df['sma_11'] < ohlc_df['sma_114']:
    print("sell")
if daily_df["Low"] <= daily_df["S2"] and open_positions and first_trade_per_day and ohlc_df['sma_11'] > ohlc_df['sma_114']:
    print("buy")
if daily_df['Low'] <= daily_df["S1"]:
    print("sell_take_profit")
if daily_df['High'] >= daily_df["R1"]:
    print("buy_take_profit")
if daily_df["High"] >= daily_df["sell_stope_loss"]:
    print("sell_stope_loss")
if daily_df["Low"] <= daily_df["buy_stope_loss"]:
    print("buy_stope_loss")